In [14]:
!pip install keras-tuner

In [15]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [16]:
fashion_mnist = keras.datasets.fashion_mnist

In [17]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [18]:
train_images = train_images/255.0
test_images = test_images/255.0

In [19]:
train_images = train_images.reshape(len(train_images), 28,28,1)
test_images = test_images.reshape(len(test_images), 28,28,1)

In [20]:
def build_model(hp):
  model = keras.Sequential([
                            keras.layers.Conv2D(
                                filters = hp.Int('conv_1_filter', min_value=64, max_value=128, step=16),
                                kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
                                activation='relu',
                                input_shape=(28,28,1)
                            ),
                            keras.layers.Conv2D(
                                filters = hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
                                kernel_size=hp.Choice('conv_2_kernel', values=[3,5]),
                                activation='relu'
                            ),
                            keras.layers.Flatten(),
                            keras.layers.Dense(
                                units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
                                activation='relu'
                            ),
                            keras.layers.Dense(10,activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy']
                )
  return model

In [21]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [22]:
tuner_search = RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='Mnist Fashion')

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fashion/tuner0.json


In [23]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [24]:
model = tuner_search.get_best_models(num_models=1)[0]

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 128)       3328      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        102432    
_________________________________________________________________
flatten (Flatten)            (None, 12800)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                1228896   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 1,335,626
Trainable params: 1,335,626
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1803 - accuracy: 0.9332 - val_loss: 0.2645 - val_accuracy: 0.9085
Epoch 5/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1449 - accuracy: 0.9460 - val_loss: 0.2626 - val_accuracy: 0.9120
Epoch 6/10
1688/1688 [==============================] - 14s 9ms/step - loss: 0.1154 - accuracy: 0.9568 - val_loss: 0.2789 - val_accuracy: 0.9113
Epoch 7/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0893 - accuracy: 0.9667 - val_loss: 0.3417 - val_accuracy: 0.9127
Epoch 8/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0721 - accuracy: 0.9726 - val_loss: 0.3528 - val_accuracy: 0.9102
Epoch 9/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0577 - accuracy: 0.9788 - val_loss: 0.4054 - val_accuracy: 0.9123
Epoch 10/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0483 - accuracy: 0.9828 - val_loss: 0.4285 - val_a